In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

Now we may compare all of our results:

In [28]:
nba_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_szn_train.csv')
nba_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_szn_test.csv')

min_minutes = nba_train[(nba_train['all_nba_c_year']==1)].MP.min()
min_G = nba_train[(nba_train['all_nba_c_year']==1)].G.min()
nba_filt_train = nba_train[(nba_train['MP']>=min_minutes) & (nba_train['G']>=min_G)]
nba_filt_test = nba_test[(nba_test['MP']>=min_minutes) & (nba_test['G']>=min_G)]

y_train = nba_filt_train['all_nba_c_year']

y_test = nba_filt_test['all_nba_c_year']

In [29]:
log_g_preds = pd.read_csv("test_preds/log_preds_g.csv")
log_f_preds = pd.read_csv("test_preds/log_preds_f.csv")
log_c_preds = pd.read_csv("test_preds/log_preds_c.csv")
log_preds_all = pd.read_csv("test_preds/log_preds_all.csv")
log_agg_preds = pd.concat([log_g_preds,log_f_preds, log_c_preds], axis = 0)


In [30]:
rf_g_preds = pd.read_csv("test_preds/rf_preds_g.csv")
rf_f_preds = pd.read_csv("test_preds/rf_preds_f.csv")
rf_c_preds = pd.read_csv("test_preds/rf_preds_c.csv")
rf_preds_all = pd.read_csv("test_preds/rf_preds_all.csv")
rf_agg_preds = pd.concat([rf_g_preds,rf_f_preds, rf_c_preds], axis = 0)

xg_g_preds = pd.read_csv("test_preds/xg_preds_g.csv")
xg_f_preds = pd.read_csv("test_preds/xg_preds_f.csv")
xg_c_preds = pd.read_csv("test_preds/xg_preds_c.csv")
xg_preds_all = pd.read_csv("test_preds/xg_preds_all.csv")
xg_agg_preds = pd.concat([xg_g_preds,xg_f_preds, xg_c_preds], axis = 0)

ens_g_preds = pd.read_csv("test_preds/ens_preds_g.csv")
ens_f_preds = pd.read_csv("test_preds/ens_preds_f.csv")
ens_c_preds = pd.read_csv("test_preds/ens_preds_c.csv")
ens_preds_all = pd.read_csv("test_preds/ens_preds_all.csv")
ens_agg_preds = pd.concat([ens_g_preds,ens_f_preds, ens_c_preds], axis = 0)

In [31]:
def pred_all_nba_df(nba_test, year):
    if year<=1988:
        return 'year must be above 1988'
    pred_df = nba_test.copy()
    #pred_df['prob_all_nba'] = model.predict_proba(nba_test)[:,1]
    pred_df = pred_df[pred_df['year']==year]
    pred_df.sort_values(by=['prob_all_nba'], ascending=False, inplace=True)
    all_nba_g = pred_df.loc[pred_df['Position']=='G','Player'].head(8)
    all_nba_f = pred_df.loc[pred_df['Position']=='F','Player'].head(8)
    all_nba_c = pred_df.loc[pred_df['Position']=='C','Player'].head(4)
    pred_team1 = pd.concat([all_nba_g.iloc[0:2], all_nba_f.iloc[0:2], all_nba_c.iloc[0:1]]).to_list()
    pred_team2 = pd.concat([all_nba_g.iloc[2:4], all_nba_f.iloc[2:4], all_nba_c.iloc[1:2]]).to_list()
    pred_team3 = pd.concat([all_nba_g.iloc[4:6], all_nba_f.iloc[4:6], all_nba_c.iloc[2:3]]).to_list()
    pred_team4 = pd.concat([all_nba_g.iloc[6:8], all_nba_f.iloc[6:8], all_nba_c.iloc[3:4]]).to_list()
    all_nba_df = pd.DataFrame({'Predicted Team 1':pred_team1, 'Predicted Team 2':pred_team2, 'Predicted Team 3':pred_team3, 'Predicted Team 4':pred_team4})
    all_nba_df.index = ['Guard', 'Guard', 'Forward', 'Forward', 'Center']
    return all_nba_df.transpose()

def true_all_nba(nba_test,year):
    teams = ['1st','2nd','3rd']
    if year<=1988:
        return 'year must be above 1988'
    year_all_nba = nba_test[(nba_test['year']==year) & (nba_test['all_nba_tm'].isin(teams))].sort_values(by=['all_nba_tm','Position'], ascending=True)
    team1 = year_all_nba.iloc[np.arange(0,5),:]['Player'].tolist()
    team2 = year_all_nba.iloc[np.arange(5,10),:]['Player'].tolist()
    team3 = year_all_nba.iloc[np.arange(10,15),:]['Player'].tolist()
    all_nba_df = pd.DataFrame({'True Team 1':team1, 'True Team 2':team2, 'True Team 3':team3})
    all_nba_df.index = ['Center', 'Forward', 'Forward', 'Guard', 'Guard']
   
    return  all_nba_df.iloc[::-1].transpose()

We have for the true All-NBA teams for 2023 as:

In [32]:
true_all_nba(nba_test, 2023)

,Guard,Guard,Forward,Forward,Center
True Team 1,Luka Dončić,Shai Gilgeous-Alexander,Jayson Tatum,Giannis Antetokounmpo,Joel Embiid
True Team 2,Stephen Curry,Donovan Mitchell,Jaylen Brown,Jimmy Butler,Nikola Jokić
True Team 3,De'Aaron Fox,Damian Lillard,LeBron James,Julius Randle,Domantas Sabonis


# Logistic Regression

### Aggregate Results (Combining 3 Positional Models)

In [33]:
print(classification_report(log_agg_preds['all_nba_c_year'], log_agg_preds['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       809
           1       0.83      0.83      0.83       120

    accuracy                           0.96       929
   macro avg       0.90      0.90      0.90       929
weighted avg       0.96      0.96      0.96       929



In [34]:
pred_all_nba_df(log_agg_preds, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Shai Gilgeous-Alexander,Donovan Mitchell,Giannis Antetokounmpo,Jayson Tatum,Joel Embiid
Predicted Team 2,Luka Dončić,Damian Lillard,LeBron James,Julius Randle,Nikola Jokić
Predicted Team 3,Stephen Curry,Trae Young,Kevin Durant,Jaylen Brown,Anthony Davis
Predicted Team 4,Ja Morant,James Harden,Jimmy Butler,Pascal Siakam,Domantas Sabonis


### Full Data Results

In [35]:
print(classification_report(log_preds_all['all_nba_c_year'], log_preds_all['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       809
           1       0.83      0.83      0.83       120

    accuracy                           0.96       929
   macro avg       0.90      0.90      0.90       929
weighted avg       0.96      0.96      0.96       929



In [36]:
pred_all_nba_df(log_preds_all, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Luka Dončić,Shai Gilgeous-Alexander,Giannis Antetokounmpo,Jayson Tatum,Joel Embiid
Predicted Team 2,James Harden,Damian Lillard,Jimmy Butler,LeBron James,Nikola Jokić
Predicted Team 3,Stephen Curry,Donovan Mitchell,Kevin Durant,Julius Randle,Anthony Davis
Predicted Team 4,Ja Morant,Trae Young,Pascal Siakam,Jaylen Brown,Domantas Sabonis


# Random Forest Results

### Aggregate Results (Combining 3 Positional Models)

In [37]:
print(classification_report(rf_agg_preds['all_nba_c_year'], rf_agg_preds['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       809
           1       0.81      0.81      0.81       120

    accuracy                           0.95       929
   macro avg       0.89      0.89      0.89       929
weighted avg       0.95      0.95      0.95       929



In [38]:
pred_all_nba_df(rf_agg_preds, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Donovan Mitchell,Luka Dončić,Jayson Tatum,Giannis Antetokounmpo,Joel Embiid
Predicted Team 2,Shai Gilgeous-Alexander,Stephen Curry,Julius Randle,Kevin Durant,Nikola Jokić
Predicted Team 3,Damian Lillard,Ja Morant,LeBron James,Jimmy Butler,Domantas Sabonis
Predicted Team 4,James Harden,Jalen Brunson,Kawhi Leonard,Jaylen Brown,Anthony Davis


### Full Model Results

In [39]:
print(classification_report(rf_preds_all['all_nba_c_year'], rf_preds_all['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       809
           1       0.78      0.78      0.78       120

    accuracy                           0.94       929
   macro avg       0.88      0.88      0.88       929
weighted avg       0.94      0.94      0.94       929



In [40]:
pred_all_nba_df(rf_preds_all, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Donovan Mitchell,Stephen Curry,Jayson Tatum,Giannis Antetokounmpo,Joel Embiid
Predicted Team 2,Shai Gilgeous-Alexander,Luka Dončić,Jimmy Butler,Kevin Durant,Nikola Jokić
Predicted Team 3,Ja Morant,De'Aaron Fox,LeBron James,Julius Randle,Anthony Davis
Predicted Team 4,Jalen Brunson,James Harden,Kawhi Leonard,Pascal Siakam,Domantas Sabonis


# XGboost Results

### Aggregate Results (Combining 3 Positional Models)

In [41]:
print(classification_report(xg_agg_preds['all_nba_c_year'], xg_agg_preds['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       809
           1       0.81      0.81      0.81       120

    accuracy                           0.95       929
   macro avg       0.89      0.89      0.89       929
weighted avg       0.95      0.95      0.95       929



In [42]:
pred_all_nba_df(xg_agg_preds, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Stephen Curry,Ja Morant,Giannis Antetokounmpo,Jayson Tatum,Joel Embiid
Predicted Team 2,Jalen Brunson,Donovan Mitchell,Julius Randle,Kevin Durant,Nikola Jokić
Predicted Team 3,Luka Dončić,James Harden,LeBron James,Jaylen Brown,Anthony Davis
Predicted Team 4,Trae Young,Darius Garland,Jimmy Butler,Pascal Siakam,Domantas Sabonis


### Full Model Results

In [43]:
print(classification_report(xg_preds_all['all_nba_c_year'], xg_preds_all['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       809
           1       0.80      0.80      0.80       120

    accuracy                           0.95       929
   macro avg       0.89      0.89      0.89       929
weighted avg       0.95      0.95      0.95       929



In [44]:
pred_all_nba_df(xg_preds_all, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Donovan Mitchell,Luka Dončić,Jayson Tatum,Giannis Antetokounmpo,Joel Embiid
Predicted Team 2,Shai Gilgeous-Alexander,Ja Morant,LeBron James,Jimmy Butler,Nikola Jokić
Predicted Team 3,Stephen Curry,Damian Lillard,Kevin Durant,Julius Randle,Anthony Davis
Predicted Team 4,Jalen Brunson,James Harden,Kawhi Leonard,Pascal Siakam,Domantas Sabonis


# Ensemble Results

### Aggregate Results (Combining 3 Positional Models)

In [45]:
print(classification_report(ens_agg_preds['all_nba_c_year'], ens_agg_preds['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       809
           1       0.81      0.81      0.81       120

    accuracy                           0.95       929
   macro avg       0.89      0.89      0.89       929
weighted avg       0.95      0.95      0.95       929



In [46]:
pred_all_nba_df(ens_agg_preds, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Donovan Mitchell,Stephen Curry,Jayson Tatum,Giannis Antetokounmpo,Joel Embiid
Predicted Team 2,Luka Dončić,Ja Morant,Julius Randle,Kevin Durant,Nikola Jokić
Predicted Team 3,James Harden,Jalen Brunson,LeBron James,Jaylen Brown,Domantas Sabonis
Predicted Team 4,Shai Gilgeous-Alexander,Damian Lillard,Jimmy Butler,Pascal Siakam,Anthony Davis


### Full Model Results

In [47]:
print(classification_report(ens_preds_all['all_nba_c_year'], ens_preds_all['pred_all_nba']))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       809
           1       0.80      0.80      0.80       120

    accuracy                           0.95       929
   macro avg       0.89      0.89      0.89       929
weighted avg       0.95      0.95      0.95       929



In [48]:
pred_all_nba_df(ens_preds_all, 2023)

,Guard,Guard,Forward,Forward,Center
Predicted Team 1,Donovan Mitchell,Luka Dončić,Jayson Tatum,Giannis Antetokounmpo,Joel Embiid
Predicted Team 2,Shai Gilgeous-Alexander,Ja Morant,LeBron James,Jimmy Butler,Nikola Jokić
Predicted Team 3,Stephen Curry,Damian Lillard,Kevin Durant,Julius Randle,Anthony Davis
Predicted Team 4,Jalen Brunson,James Harden,Kawhi Leonard,Pascal Siakam,Domantas Sabonis


In [49]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc

auc_scores = [auc(*roc_curve(log_agg_preds['pred_all_nba'], log_agg_preds['prob_all_nba'])[:2]),
                auc(*roc_curve(rf_agg_preds['pred_all_nba'], rf_agg_preds['prob_all_nba'])[:2]),
                auc(*roc_curve(xg_agg_preds['pred_all_nba'], xg_agg_preds['prob_all_nba'])[:2]),
                auc(*roc_curve(ens_agg_preds['pred_all_nba'], ens_agg_preds['prob_all_nba'])[:2])]


#Putting precision and recall and ROC-AUC score into a dataframe for all 4 models
model_names = ['Logistic Regression', 'Random Forest', 'XGBoost', 'Ensemble']
precision = [0,0,0,0]
recall = [0,0,0,0]
roc_auc = [0,0,0,0]
f1_scores = [0,0,0,0]
accuracy = [0,0,0,0]
for i, preds in enumerate([log_agg_preds, rf_agg_preds, xg_agg_preds, ens_agg_preds]):
    precision[i] = precision_score(preds['all_nba_c_year'], preds['pred_all_nba'])
    recall[i] = recall_score(preds['all_nba_c_year'], preds['pred_all_nba'])
    roc_auc[i] = auc_scores[i]
    f1_scores[i] = f1_score(preds['all_nba_c_year'], preds['pred_all_nba'])
    accuracy[i] = accuracy_score(preds['all_nba_c_year'], preds['pred_all_nba'])
res_df = pd.DataFrame({'Model': model_names, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F-1 Score':f1_scores ,'ROC-AUC': roc_auc})
res_df.to_csv('res/results_agg.csv', index=False)
res_df

,Model,Accuracy,Precision,Recall,F-1 Score,ROC-AUC
0,Logistic Regression,0.956943,0.833333,0.833333,0.833333,0.990678
1,Random Forest,0.950484,0.808333,0.808333,0.808333,0.988000
2,XGBoost,0.950484,0.808333,0.808333,0.808333,0.982118
3,Ensemble,0.950484,0.808333,0.808333,0.808333,0.993665


In [50]:
pd.read_csv('res/results_all.csv')

,Model,Accuracy,Precision,Recall,F-1 Score,ROC-AUC
0,Logistic Regression,0.956943,0.833333,0.833333,0.833333,0.975371
1,Random Forest,0.944026,0.783333,0.783333,0.783333,0.972930
2,XGBoost,0.948332,0.800000,0.800000,0.800000,0.975762
3,Ensemble,0.948332,0.800000,0.800000,0.800000,0.974742
